In [6]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute, Aer
from qiskit.utils import QuantumInstance
from qiskit.opflow import PauliExpectation, CircuitSampler, StateFn, X, Y, Z, I, CircuitStateFn 
import numpy as np

#function to change bool String to integer
def boolStringToInt(array):
    integer = 0
    for i in range(len(array)):
        if (array[i] == "1"):
            integer += 2**i
    return integer

# Measures the expectation value more effeciently by grouping certain Paulis
def rotateAndMeasure(pauliOperator, stateFunc):
    # as of now, pauliOperator should be type PauliSumOp from qiskit
    # stateFunc is the circuit that represents ansatz
    
    op = pauliOperator
    state = stateFunc
    state = CircuitStateFn(state)
    
    backend = Aer.get_backend('qasm_simulator') 
    qInstance = QuantumInstance(backend, shots=1024)
    measurableExp = StateFn(op, is_measurement=True).compose(state) 
    
    expectationVal = PauliExpectation().convert(measurableExp)
    sampler = CircuitSampler(qInstance).convert(expectationVal) 
    
    return sampler.eval().real  

def VQE(numberOfQubits, theta, standardError, pauliOperator):
    #theta is an initial angle
    #target standard error is given to function
    #function should return expectationvalue for single angle
    register = QuantumRegister(numberOfQubits)
    circuit = QuantumCircuit(register)
    expectationValue = 0.00
    measured = []
    lastMin = 0
    minExp = 1
    while (lastMin < 100):
        for i in range(numberOfQubits^4):
            for i in range((1/standardError)^2):
                #prepare state function of theta[i]
                for qubit in register:
                    circuit.rx(theta, qubit)
                    circuit.ry(theta, qubit)

                #entangled anzatz states preparation
                for k in range(len(register) - 1):
                    circuit.cx(register[k], register[k+1])
                
                #measures expectation value, adds to list, compares with min
                exp = rotateAndMeasure(pauliOperator, circuit)
                measured.append(exp)
                if exp < minExp:
                    minExp = exp
                    lastMin = 0
                else:
                    lastMin += 1
    #gets rid of all expectation values after the minimum value
    upToMin = measured[:len(measured)-lastMin]
    
    #adds up the remaining expectation values
    expectationValue = sum(upToMin)
    return expectationValue

# backup measeurement algorithm for energy
def measureExpectationValue(array, standardError, circuit):
    totalH = 0.00
    counter = 0

    #simulation of measurement results
    measurement = ClassicalRegister(len(array))
    circuit.measure(input, measurement)
    simulator = Aer.get_backend('aer_simulator')

    #runs 1/standard error squared times for standard error given
    simulation = execute(circuit, simulator, shots=(1/standardError)^2)
    mresult = simulation.result()
    counts = mresult.get_counts(circuit)
    for(measured_state, count) in counts.items():
        counter += count
        intM = boolStringToInt(measured_state)
        totalH += intM * count
    #Find expectationValue for given set of Pauli Strings for energies                
    expectationValue = totalH / counter   
    return expectationValue 